In [1]:
import pandas as pd
from soil_fertility.components.data_ingestion import DataIngestion
from soil_fertility.components.data_transformation import DataTransformation

In [2]:
obj = DataIngestion()
part,train_path,test_path= obj.init_ingestion(
    "/home/redha/Documents/projects/NLP/datamining project/Soil-Fertility/data/Dataset1.csv"
)


In [3]:
tranformation_obj = DataTransformation()
part,preprocessor,processed_train_path,processed_test_path = tranformation_obj.transform(
    train_path,test_path,target="Fertility"
)

In [5]:
from sklearn.preprocessing import MinMaxScaler 
df_train=pd.read_csv(train_path)
df_test=pd.read_csv(test_path)



In [7]:
df_train.drop(['Fertility'],axis=1,inplace=True)
df_test.drop(['Fertility'],axis=1,inplace=True)

In [8]:
scaler = MinMaxScaler()
ss=scaler.fit_transform(df_train)
ss1=scaler.transform(df_test)


In [4]:
preprocessor

ColumnTransformer(transformers=[('numerical',
                                 Pipeline(steps=[('imputer',
                                                  MinMaxTransformation())]),
                                 Index(['N', 'P', 'K', 'pH', 'EC', 'OC', 'S', 'Zn', 'Fe', 'Cu', 'Mn', 'B',
       'OM'],
      dtype='object'))])

In [25]:
df=pd.read_excel('/home/redha/Documents/projects/NLP/datamining project/Soil-Fertility/data/Dataset3.xlsx')


In [65]:
from sklearn.base import BaseEstimator, TransformerMixin
from typing import List
class EqualWidthDescritizer(BaseEstimator, TransformerMixin):
    def __init__(self, k: int) -> None:
        self.k = k
        self.bin_edges = None 
        

    

    def assigning_classes_equal_width(self, df : pd.DataFrame, column : str) -> List[int]:
        """ this function assign classes to the dataframe column based on the number of classes k with equal width

        Args:
            df (pd.DataFrame): dataframe to assign classes to
            column (str): column to assign classes to
            k (int): number of classes

        Returns:
            List[int]: list of classes assigned to each row in the dataframe
        """

        new_df=df.copy()        

        class_assignments = []
        current_class = 1
        current_boundary = self.sorted_data.min() + self.bin_width            

        for observation in self.sorted_data:
            if observation > current_boundary:
                current_class += 1
                current_boundary += self.bin_width

                if current_boundary > self.sorted_data.max():
                    current_boundary = self.sorted_data.max()
                    current_class = current_class-1

            class_assignments.append(current_class)

        new_df[column] = class_assignments


        return new_df
    
    def fit(self, X, y=None):
        values=X.iloc[:,0]
        self.sorted_data = values.sort_values()
        self.range_value = self.sorted_data.max() - self.sorted_data.min()
        self.bin_width = self.range_value / self.k
        
        self.bin_edges_ = [self.sorted_data.min() + i * self.bin_width for i in range(1, self.k)]

        return self
    
    def transform(self, X, y=None, **fit_params):

        if self.bin_edges_ is None:
            raise ValueError("fit() method must be called before transform()")

        # Use the saved bin edges to discretize the data
        class_assignments = pd.cut(X.iloc[:,0], bins=self.bin_edges_, labels=False, include_lowest=True)
        
        X.iloc[:,0]=class_assignments

        return X

    def get_feature_names_out(self):
        pass

In [66]:
df=pd.read_excel('/home/redha/Documents/projects/NLP/datamining project/Soil-Fertility/data/Dataset3.xlsx')

In [68]:
#import pipeline
from sklearn.pipeline import Pipeline
#import column transformer
from sklearn.compose import ColumnTransformer

test_pipe=Pipeline(steps=[
    ('equal_width',EqualWidthDescritizer(5)),
])

col_tran=ColumnTransformer(transformers=[
    ('equal_width',test_pipe,['Temperature']),
],remainder='passthrough')

r=col_tran.fit_transform(df)




In [71]:
r=pd.DataFrame(r)

In [77]:
r.iloc[:,0].unique()

array([1.0, nan, 2.0, 0.0], dtype=object)

In [58]:
r

array([[1, 82.84, 295.61, 'Clayey', 'rice', 'DAP'],
       [1, 96.65, 178.96, 'laterite', 'Coconut', 'Good NPK'],
       [1, 81.64, 270.44, 'silty clay', 'rice', 'MOP'],
       ...,
       [5, 83.54, 230.45, 'Clayey', 'rice', 'MOP'],
       [5, 83.47, 217.38, 'Clayey', 'rice', 'MOP'],
       [5, 84.81, 298.56, 'silty clay', 'rice', 'MOP']], dtype=object)

In [ ]:
TypeError: DataFrame.sort_values() missing 1 required positional argument: 'by'


In [ ]:
        sorted_data = X.sort_values()
        range_value = sorted_data.max() - sorted_data.min()
        bin_width = range_value / self.k

        # Calculate bin edges based on the training set
        self.bin_edges_ = [sorted_data.min() + i * bin_width for i in range(1, self.k)]